<a href="https://colab.research.google.com/github/ernesto-butto/ag2-workshop-simulation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar AG2
https://docs.ag2.ai/docs/home/quick-start

In [2]:
!pip install ag2

In [3]:
# 1. Import our agent class
from autogen import ConversableAgent

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
